In [56]:
ex1 = """start-A
start-b
A-c
A-b
b-d
A-end
b-end""".splitlines()

def parse(inp):
    g = [tuple(l.strip().split('-')) for l in inp]
    caverns: dict[str, Cavern] = {}
    for c in g:
        if c[0] not in caverns:
            caverns[c[0]] = Cavern(c[0])
        if c[1] not in caverns:
            caverns[c[1]] = Cavern(c[1])
        caverns[c[0]].add(caverns[c[1]])
        caverns[c[1]].add(caverns[c[0]])
    return caverns
caverns = parse(ex1)
caverns

{'start': start, 'A': A, 'b': b, 'c': c, 'd': d, 'end': end}

In [57]:
class Cavern():
    def __init__(self, name: str):
        self.name = name
        self.neighbors: set[Cavern] = set()
        self.visits = 0
    def __repr__(self) -> str:
        # n = ",".join([n.name for n in self.neighbors])
        # return f"Cavern: {self.name}: {n}"
        return self.name

    def is_large(self):
        return self.name.upper() == self.name

    def add(self, neighbor):
        self.neighbors.add(neighbor)

    def can_visit(self):
        if self.is_large():
            return True
        if self.name == "start":
            return False
        return self.visits == 0
    def end(self):
        return self.name == "end"
    def visit(self):
        self.visits += 1
    def reset(self):
        self.visits = 0



In [58]:
from copy import deepcopy

In [59]:
def show(caverns: dict[str, Cavern]):
    for c in sorted(caverns.keys()):
        n = ",".join(sorted([n.name for n in caverns[c].neighbors]))
        print(f"[{caverns[c].name}]: {n}")

In [61]:
def traverse(caverns: dict[str, Cavern], cur: str, cur_path: list[Cavern], all_paths: list[list[Cavern]]):
    new_branches: list[str] = []
    for n in caverns[cur].neighbors:
        if n.can_visit():
            new_branches.append(n.name)
    if len(new_branches) == 0:
        return all_paths
    #print(cur, new_branches)

    for c in new_branches:
        updated_path = cur_path.copy()
        updated_path.append(c)
        if caverns[c].end():
            all_paths.append(updated_path)
            continue
        new_caverns = deepcopy(caverns)
        new_caverns[c].visit()
        all_paths = traverse(new_caverns, c, updated_path, all_paths)

    return all_paths

paths = traverse(caverns, "start", ["start"], [])
len(paths)


10

In [62]:
def part1(caverns):
    paths = traverse(caverns, "start", ["start"], [])
    return len(paths)

part1(parse(ex1))

10

In [63]:
ex2 = """dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc""".splitlines()
part1(parse(ex2))

19

In [64]:
ex3 = """fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW""".splitlines()
part1(parse(ex3))

226

In [66]:
pzl = open("data/12.txt").readlines()
part1(parse(pzl))

5576

## Part 2

In [81]:
class Cavern():
    def __init__(self, name: str):
        self.name = name
        self.neighbors: set[Cavern] = set()
        self.visits = 0
        self.large = self.name.upper() == self.name
    def __repr__(self) -> str:
        # n = ",".join([n.name for n in self.neighbors])
        # return f"Cavern: {self.name}: {n}"
        return self.name

    def add(self, neighbor):
        self.neighbors.add(neighbor)

    def can_visit(self, twice):
        if self.large:
            return True
        if self.name == "start":
            return False
        if twice:
            return self.visits == 0
        return self.visits < 2
    def end(self):
        return self.name == "end"
    def visit(self):
        self.visits += 1
        if self.large:
            return False
        return self.visits == 2
    def reset(self):
        self.visits = 0

def traverse(caverns: dict[str, Cavern], cur: str, cur_path: str, all_paths: list[str], twice: bool):
    new_branches: list[str] = []
    for n in caverns[cur].neighbors:
        if n.can_visit(twice):
            new_branches.append(n.name)
    if len(new_branches) == 0:
        return all_paths
    #print(cur, new_branches)

    for c in new_branches:
        updated_path = cur_path + "," + c
        if caverns[c].end():
            all_paths.append(updated_path)
            continue
        new_caverns = deepcopy(caverns)
        new_twice = new_caverns[c].visit()
        all_paths = traverse(new_caverns, c, updated_path, all_paths, twice or new_twice)

    return all_paths


paths = traverse(parse(ex1), "start", "start", [], False)
len(paths)


36

In [82]:
def part2(caverns):
    paths = traverse(caverns, "start", "start", [], False)
    return len(paths)

print(part2(parse(ex1)))
print(part2(parse(ex2)))
print(part2(parse(ex3)))

36
103
3509


In [77]:
print(part2(parse(pzl)))

152837


## Faster!

In [83]:
from collections import defaultdict
def parse(inp):
    g = [tuple(l.strip().split('-')) for l in inp]
    caverns = defaultdict(set)
    for c in g:
        caverns[c[0]].add(c[1])
        caverns[c[1]].add(c[0])
    return caverns
caverns = parse(ex1)
caverns

defaultdict(set,
            {'start': {'A', 'b'},
             'A': {'b', 'c', 'end', 'start'},
             'b': {'A', 'd', 'end', 'start'},
             'c': {'A'},
             'd': {'b'},
             'end': {'A', 'b'}})

In [107]:
def small(c: str):
    return c.lower() == c


def traverse(caverns: dict[str, set], cur: str, cur_path: set[str], twice: bool):
    res = 0
    for n in caverns[cur]:
        if n == "end":
           res += 1
           continue 

        ok = False
        new_t = twice
        if twice and n != "start" and small(n) and n in cur_path: 
            new_t = False
            ok = True
        elif not small(n) or n not in cur_path:
            ok = True
        if not ok:
            continue
        updated_path = cur_path | {n}
        res += traverse(caverns, n, updated_path, new_t)

    return res


def part21(caverns):
    return traverse(caverns, "start", set(["start"]), True)

print(part21(parse(ex1)))
print(part21(parse(ex2)))
print(part21(parse(ex3)))

36
103
3509


In [108]:
print(part21(parse(pzl)))

152837
